In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import time

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

data = requests.get(standings_url)

time.sleep(5) # adding a delay to avoid overloading the server

data.text

In [ ]:
soup = BeautifulSoup(data.text)

standings_table = soup.select('table.stats_table')[0]

if not standings_table:
    print("No standings table found.")
else:
    standings_table = soup.select('table.stats_table')[0]

The code uses BeautifulSoup to parse the HTML content stored in `data.text`. It then attempts to select the first table with the class `stats_table`. If no standings table is found, it prints a message; otherwise, it assigns the selected table to the variable `standings_table`. (The if statement is added because I kept running into errors because the table index was out of bounds due to me sending too many requests)


In [ ]:
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/squads/' in l]

links

team_urls = [f"https://fbref.com{l}" for l in links] ## formatting back to links

The code retrieves all anchor tags from the `standings_table` to find hyperlinks. It then extracts the `href` attributes from these tags to create a list of URLs. The list is filtered to keep only those links that contain `'/squads/'`, which are team-specific pages. Finally, it constructs full URLs for each team by adding the base URL `https://fbref.com` to the filtered links.

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

The code iterates over a list of years to scrape Premier League match data for each team. It fetches the standings data and extracts team URLs from the standings table. The code then updates the standings URL to point to the previous season's data. For each team, it retrieves match and shooting statistics, merges the datasets, and filters for Premier League matches. Then it appends the processed data to a list while adding a delay between requests so it doesn't overload the server.


In [ ]:
stat_df = pd.concat(all_teams) 
stat_df.to_csv("matches.csv") ## importing to csv